In [0]:
! pip install geopandas
! pip install geodatasets
! pip install folium matplotlib mapclassify contextily

In [0]:
import sys
basePath=''
gistPath=''
sys.path.append(basePath)
import dataBricksConfig as db

In [0]:
import numpy as np
import pandas as pd
import os
import gistMCLive as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import contextily as cx

#Well Listings and QC

##Load selected wells and cumulative volumes

In [0]:
basePath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/'
prefix=basePath+'Stanton11-4-23_'
dPPWellDF=pd.read_csv(prefix+'DeepPPWells.csv')
dPEWellDF=pd.read_csv(prefix+'DeepPEWells.csv')
sPPWellDF=pd.read_csv(prefix+'ShallowPPWells.csv')
sPEWellDF=pd.read_csv(prefix+'ShallowPEWells.csv')
EQDF=pd.read_csv(basePath+'/11-4-23EQ.csv')
EQDF=geopandas.GeoDataFrame(EQDF, geometry=geopandas.points_from_xy(EQDF['Longitude'],EQDF['Latitude']), crs="EPSG:4326")

##Plot Deep and Shallow well selection

In [0]:
print(dPPWellDF['DDRatio'].min(),dPPWellDF['DDRatio'].max(),dPPWellDF['TotalBBL'].min(),dPPWellDF['TotalBBL'].max())
print(dPEWellDF['DDRatio'].min(),dPEWellDF['DDRatio'].max(),dPEWellDF['TotalBBL'].min(),dPEWellDF['TotalBBL'].max())
dPPWellDF['MMBBL']=dPPWellDF['TotalBBL']/1000000.
dPEWellDF['MMBBL']=dPEWellDF['TotalBBL']/1000000.

In [0]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(figsize=(36,24))
plt.title('Deep wells')
divider = make_axes_locatable(ax)
PPWellsGDF = geopandas.GeoDataFrame(dPPWellDF, geometry=geopandas.points_from_xy(dPPWellDF['SurfaceHoleLongitude'], dPPWellDF['SurfaceHoleLatitude']), crs="EPSG:4326")
PEWellsGDF = geopandas.GeoDataFrame(dPEWellDF, geometry=geopandas.points_from_xy(dPEWellDF['SurfaceHoleLongitude'], dPEWellDF['SurfaceHoleLatitude']), crs="EPSG:4326")
xlim=(min(PEWellsGDF['SurfaceHoleLongitude'])-0.1,max(PEWellsGDF['SurfaceHoleLongitude'])+0.1)
ylim=(min(PEWellsGDF['SurfaceHoleLatitude'])-0.1,max(PEWellsGDF['SurfaceHoleLatitude'])+0.1)
#PEWellsGDF.plot(ax=ax,column='DDRatio',marker='.',vmin=0.,vmax=6.,markersize='MMBBL',legend=True)
PPWellsGDF.plot(ax=ax,column='DDRatio',marker='x',vmin=0.,vmax=1.,markersize='MMBBL',legend=True)
EQDF.plot(ax=ax,color='purple',marker='*',markersize=300)
plt.xlim(xlim)
plt.ylim(ylim)
cx.add_basemap(ax,zoom=10,crs="EPSG:4326")
plt.show()

# Make zoomed in version
fig, ax = plt.subplots(figsize=(24,24))
plt.title('Deep wells')
divider = make_axes_locatable(ax)
PPWellsGDF = geopandas.GeoDataFrame(dPPWellDF, geometry=geopandas.points_from_xy(dPPWellDF['SurfaceHoleLongitude'], dPPWellDF['SurfaceHoleLatitude']), crs="EPSG:4326")
PEWellsGDF = geopandas.GeoDataFrame(dPEWellDF, geometry=geopandas.points_from_xy(dPEWellDF['SurfaceHoleLongitude'], dPEWellDF['SurfaceHoleLatitude']), crs="EPSG:4326")
#PEWellsGDF.plot(ax=ax,column='DDRatio',marker='.',vmin=0.,vmax=6.,markersize='MMBBL',legend=True)
PPWellsGDF.plot(ax=ax,column='DDRatio',marker='x',vmin=0.,vmax=1.,markersize='MMBBL',legend=True)
xlim=(min(EQDF['Longitude'])-0.2,max(EQDF['Longitude'])+0.2)
ylim=(min(EQDF['Latitude'])-0.2,max(EQDF['Latitude'])+0.2)
EQDF.plot(ax=ax,color='purple',marker='*',markersize=300)
plt.xlim(xlim)
plt.ylim(ylim)
cx.add_basemap(ax,zoom=12,crs="EPSG:4326")
plt.show()

# Load Data


In [0]:

dWellPPScenarios=pd.read_csv('/gist/Outputs/Stanton11-4-23_v2_'+'deepPP.csv')
dWellPEScenarios=pd.read_csv('/gist/Outputs/Stanton11-4-23_v2_'+'deepPE.csv')
sWellPPScenarios=pd.read_csv('/gist/Outputs/Stanton11-4-23_v2_'+'shallowPP.csv')
sWellPEScenarios0=pd.read_csv('/gist/Outputs/Stanton11-4-23_v2_'+'shallowPE0.csv')
sWellPEScenarios1=pd.read_csv('/gist/Outputs/Stanton11-4-23_v2_'+'shallowPE1.csv')

In [0]:
sWellPEScenarios=pd.concat([sWellPEScenarios0,sWellPEScenarios1])
dWellPPScenarios.info()
dWellPEScenarios.info()
sWellPPScenarios.info()
sWellPEScenarios.info()

#Generate Realization Plots

In [0]:

# Make subset of each dataframe with one row per realization
dPPReal=dWellPPScenarios.groupby('Realization').first()[['NumWells','TotalPressure']]
dPEReal=dWellPEScenarios.groupby('Realization').first()[['NumWells','TotalStress','Theta']]
sPPReal=sWellPPScenarios.groupby('Realization').first()[['NumWells','TotalPressure']]
sPEReal=sWellPEScenarios.groupby('Realization').first()[['NumWells','TotalStress','Theta']]

In [0]:
# Population plots - what choices do I make here? - these should be per-realization statistics
#   Histograms of total stresses, pressures
#   Histograms of numbers of wells contributing
#   Histograms of slip tendency direction - aka rake
fig,axes=plt.subplots(2,4, sharey=True, figsize=(10,10))
axes[0,1].set_title('Shallow Pressure')
axes[0,0].set_title('Deep Pressure')
axes[0,3].set_title('Shallow Stress')
axes[0,2].set_title('Deep Stress')
sns.histplot(ax=axes[0,0],x=dPPReal.TotalPressure)
sns.histplot(ax=axes[0,1],x=sPPReal.TotalPressure)
sns.histplot(ax=axes[0,2],x=dPEReal.TotalStress)
sns.histplot(ax=axes[0,3],x=sPEReal.TotalStress)
sns.histplot(ax=axes[1,0],x=dPPReal.NumWells)
sns.histplot(ax=axes[1,1],x=sPPReal.NumWells)
sns.histplot(ax=axes[1,2],x=dPEReal.NumWells)
sns.histplot(ax=axes[1,3],x=sPEReal.NumWells)
axes[0,0].set_xlabel('Pressure (psi)')
axes[0,1].set_xlabel('Pressure (psi)')
axes[0,2].set_xlabel('Stress (psi)')
axes[0,3].set_xlabel('Stress (psi)')
axes[1,0].set_xlabel('Number of wells')
axes[1,1].set_xlabel('Number of wells')
axes[1,2].set_xlabel('Number of wells')
axes[1,3].set_xlabel('Number of wells')



fig,axes=plt.subplots(1,2, sharey=True, figsize=(5,5))
axes[1].set_title('Shallow Predicted Rakes')
axes[0].set_title('Deep Predicted Rakes')
sns.histplot(ax=axes[0],x=dPEReal.Theta)
sns.histplot(ax=axes[1],x=sPEReal.Theta)
axes[0].set_xlabel('Rake (deg)')
axes[1].set_xlabel('Rake (deg)')
# Injection plots
print(EQDF.Rake)
# Correct well injections

#Relative Contribution Ranges
## Deep

In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
#print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
#print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))
maxPressureList=[]
maxPressureDF=pd.DataFrame(columns=['Name','ID','MaxPressure'])
names=[]
ids=[]
maxps=[]
for ID in deepWindowedWells:
  maxps.append(max(filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures']))
  names.append(filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Name'].iloc[0])
  ids.append(ID)
  #print(ID,name,maxPressure)
  #nReal=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID].shape[0]
  #maxPressureList.extend([maxPressure,]*nReal)
#filtDWellPPScenarios['MaxPressure']=maxPressureList
#filtDWellPPScenarios=filtDWellPPScenarios.sort_values(by='MaxPressure',ascending=False)
#print(filtDWellPPScenarios)
maxPressureDict={'Name': names, 'ID':ids, 'MaxPressure': maxps} 
maxPressureDF=pd.DataFrame(maxPressureDict).sort_values(by='MaxPressure',ascending=False)
#filtDWellPPScenarioOrder=filtDWellPPScenarios[filtDWellPPScenarios['Realization']==0].sort_values(by='MaxPressure',ascending=False)
print(maxPressureDF)

winMaxPressureDF=maxPressureDF.iloc[0:13]
winDWellPPScenariosDF=filtDWellPPScenarios[filtDWellPPScenarios['ID'].isin(winMaxPressureDF['ID'])]
print(winMaxPressureDF)

In [0]:
OLRPWells=['']
NonOLRPWells=['']

OLRPWellPPScenariosDF=filtDWellPPScenarios[filtDWellPPScenarios['Name'].isin(OLRPWells+NonOLRPWells)]

In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(20,10))
sns.set(font_scale=2.5)
# Relative contributions
sns.stripplot(data=OLRPWellPPScenariosDF,x='Pressures',y='Name',dodge=True,jitter=True,alpha=0.7,linewidth=0,edgecolor='white',ax=axes,order=OLRPWells+NonOLRPWells,size=25)
axes.set_title('Deep Well Pressures',fontsize=25)
axes.set_xlabel('Pressure Increase (PSI)',fontsize=25)
axes.set_ylabel('Well Name',fontsize=25)

#Plot Shallow / Deep injection histories of selected wells

In [0]:
#What corrections need to be made?
# Load deep injection data from selected wells
dWellInjectionDF=pd.read_csv('/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/Stanton11-4-23_DeepPPInj.csv')
# Limit injection to wells in contributors list
newWellList=OLRPWells+NonOLRPWells
print(newWellList)
newWellIDList=dPPWellDF[dPPWellDF['WellName'].isin(newWellList)]['ID']
dContributorInjectionDF=dWellInjectionDF[dWellInjectionDF['ID'].isin(newWellIDList)]
# Limit injection to wells with DDRatio less than 0.25
selectedWellIDList=dPPWellDF[dPPWellDF['DDRatio']<0.25]['ID']
dSelectedInjectionDF=dWellInjectionDF[dWellInjectionDF['ID'].isin(selectedWellIDList)]


In [0]:
startDate=pd.to_datetime('01-01-2019')
endDate=pd.to_datetime('11-04-2023')
title='Deep OLRP well injection history'
legend=True
#injDF,wellDF,startDate,endDate,title,legend):
fig, ax = plt.subplots(figsize=(24,12))
# Merge well information onto injection rates
# Need to incorporate time sampling other than 10 day
injQC=dContributorInjectionDF.merge(dPPWellDF,on='ID')
injQC['Date']=pd.to_datetime(injQC.Date)
# Generate x and y
dates=injQC['Date'].drop_duplicates()
# Loop over wells
oDate=min(dates)
eDate=max(dates)
dateRange=pd.date_range(start=oDate,end=eDate, freq='10D')
# Get x min, x max
wellIDs=injQC['ID'].drop_duplicates()
allWellBPDs=[]
allWellLabels=[]
for well in wellIDs:
  wellDates=injQC[injQC['ID']==well]['Date']
  wellBPDs=injQC[injQC['ID']==well]['BPD']
  minWD=min(wellDates)
  maxWD=max(wellDates)
  nPrepend=sum(dateRange<minWD)
  nAppend=sum(dateRange>maxWD)
  #print('Well ',well,' ',minWD,maxWD,oDate,eDate,nPrepend,' prePad, ',nAppend,' postPad, ',len(dateRange))
  #print('allWellBPDs: ',allWellBPDs)
  if nPrepend>0:
    prependZeroBPDs=pd.Series(np.zeros([nPrepend]))
  else:
    prependZeroBPDs=None
  if nAppend>0:
    appendZeroBPDs=pd.Series(np.zeros([nAppend,1]))
  else:
    appendZeroBPDs=None
  padWellBPDs=pd.concat([prependZeroBPDs,wellBPDs,appendZeroBPDs])
  allWellBPDs.append(padWellBPDs)
  #allWellLabels.append(injQC['WellName'][injQC['ID']==well][0])
  allWellLabels.append(injQC['WellName'][injQC['ID']==well].iloc[0])
# Now have allWellBPDs, allWellLabels, and dateRange all set
# Generate Stacked area plot of injection
sns.set_theme()
# y needs to be defined for each well, and extended for the full time series
col = sns.color_palette("hls", len(wellIDs))
#zoomDate=oneEQ['Origin Date'] - pd.DateOffset(years=2)
ax.stackplot(dateRange,allWellBPDs, labels=allWellLabels,colors=col)
ax.set_title(title)
ax.set_xlabel('Date')
ax.set_ylabel('Rate (BPD)')
if startDate!=None or endDate!=None: plt.xlim((startDate,endDate))
if legend: plt.legend(loc='upper left')
plt.show()

In [0]:
# Loop over realizations and plot pressure histories - start with one
fileName='Stanton11-4-23_v2_deepPPTimeSeries3601 SWD 1.npy'
oneWellTimeSeries=np.load(basePath+fileName)
times=np.load(basePath+'/Stanton11-4-23_v2_deepPPTimes.npy')
print(times.shape,oneWellTimeSeries.shape)

In [0]:
# Make a big dataframe with all of the realizations for all wells

# Form time and realization arrays

for wellName in OLRPWells:
  # Load .npy file for time series
  filePath=basePath+'Stanton11-4-23_v2_deepPPTimeSeries'+wellName+'.npy'
  oneWellTimeSeries=np.load(filePath)
  # Now plot per-well pressures
  n0=oneWellTimeSeries.shape[0]
  n1=oneWellTimeSeries.shape[1]
  timesA=np.reshape(times[:-1],[1582,1])
  timesArray=np.repeat(timesA,n1,axis=1)
  real=np.repeat(np.arange(0,500,1),n0,axis=0)
  print(wellName,n1,oneWellTimeSeries.size,timesArray.size,real.size)
  # Make dataframe with columns - Date, Pressure, Realization
  data = {'Pressure': oneWellTimeSeries.flatten('F'), 'Days': timesArray.flatten('F'), 'Realization':real}
  pDF=pd.DataFrame.from_dict(data)
  pDF['Date']=pd.to_datetime('01-01-1970') + pd.to_timedelta(pDF['Days'],"d")
  fig, ax = plt.subplots(figsize=(24,12))
  sns.lineplot(data=pDF,x='Date',y='Pressure',hue='Realization',estimator=None,sort=False)
  plt.xlim((startDate,endDate))
  ax.set_title(wellName+' Pressure Realizations')

In [0]:
# Now plot per-well pressures
n0=oneWellTimeSeries.shape[0]
n1=oneWellTimeSeries.shape[1]
timesA=np.reshape(times[:-1],[1582,1])
timesArray=np.repeat(timesA,n1,axis=1)
real=np.repeat(np.arange(0,500,1),n0,axis=0)
print(n1,oneWellTimeSeries.size,timesArray.size,real.size)
# Make dataframe with columns - Date, Pressure, Realization
data = {'Pressure': oneWellTimeSeries.flatten('F'), 'Days': timesArray.flatten('F'), 'Realization':real}
pDF=pd.DataFrame.from_dict(data)
pDF['Date']=pd.to_datetime('01-01-1970') + pd.to_timedelta(pDF['Days'],"d")

In [0]:

fig, ax = plt.subplots(figsize=(24,12))
sns.lineplot(data=pDF,x='Date',y='Pressure',hue='Realization',estimator=None,sort=False)
plt.xlim((startDate,endDate))
ax.set_title('3601 SWD Pressure Realizations')

#plt.plot(timesArray[1350:,:], oneWellTimeSeries[1350:,:])
#plt.show()

In [0]:
# Make stacked volumes plot of these two sets of injection data
# Start Day is 2010
startDay=14610
dD=10
import datetime
epoch = datetime.date(1970,1,1)
nWellsOLRP=len(newWellIDList)
maxDay=dContributorInjectionDF['Days'].max()
nD=1+int((maxDay-startDay)/dD)
daysVec=range(startDay,startDay+(nD-1)*dD,dD)
dateVec=[]
for iD in daysVec:
  dateVec.append(epoch+ datetime.timedelta(days=iD))
injArray=np.zeros([nWellsOLRP,nD])
nameList=[]
i1=0
print(maxDay,nD,startDay,nWellsOLRP)
for wellID in newWellIDList:
  injDF=dContributorInjectionDF[dContributorInjectionDF['ID']==wellID]
  print(wellID,injDF)
  inputMinDay=min(injDF.Days)
  inputMaxDay=max(injDF.Days)
  injNP=injDF.BPD.to_numpy()
  # si0 - start of input
  # ei0 - end of input
  # so0 - start of output - relative to startDay
  # eo0 - end of output
  if inputMinDay>=startDay:
    # Case 1 - the injection data starts after the beginning of the plot
    si0=0
    # Get maximum
    ei0=int((inputMaxDay-inputMinDay)/dD)
    # Get output start and end
    so0=int((inputMinDay-startDay)/dD)
    eo0=int((inputMaxDay-startDay)/dD)
  elif inputMinDay<startDay:
    # Case 2 - the injection data starts before the beginning of the plot
    si0=int((startDay-inputMinDay)/dD)
    ei0=int((inputMaxDay-inputMinDay)/dD)
    so0=0
    eo0=int((inputMaxDay-startDay)/dD)
  print(i1,si0,ei0,so0,eo0)
  # I do not check the end date, assume they are all the same
  injArray[i1,so0:eo0]=injNP[si0:ei0]
  nameList.append(dPPWellDF['WellName'][dPPWellDF['ID']==wellID])
  i1=i1+1
fig, ax = plt.subplots(figsize=(12,24))
#print(dateVec)
print(injArray)
plt.stackplot(x=dateVec,data=injArray,labels=nameList)
ax.set_title('Injection')
ax.set_xlabel('Days')
ax.set_ylabel('Rate (BPD)')

In [0]:
#Plot Shallow / Deep pressure histories on a per-well basis

In [0]:
# All wells
fig,axes=plt.subplots(1,1, sharex=True, figsize=(20,40))
sns.set(font_scale=2.5)
# Relative contributions
sns.stripplot(data=filtDWellPPScenarios,x='Pressures',y='Name',dodge=True,jitter=True,alpha=0.7,linewidth=0,edgecolor='white',ax=axes,order=maxPressureDF,size=25)
axes.set_title('Deep Well Pressures',fontsize=25)
axes.set_xlabel('Pressure (PSI)',fontsize=25)
axes.set_ylabel('Well Name',fontsize=25)

In [0]:
# Reduce number of rows to wells with selected names
WellNames=['',]
filtDWellPPScenariosWin=filtDWellPPScenarios[filtDWellPPScenarios['Name'].isin(WellNames)]

In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(10,5))
# Relative contributions
sns.stripplot(data=filtDWellPPScenariosWin,x='Pressures',y='Name',dodge=True,jitter=True,alpha=0.1,linewidth=2,edgecolor='white',ax=axes)
axes.set_title('Deep Well Pressures')
axes.set_xlabel('Pressure (PSI)')

## Shallow Relative Contribution Ranges

In [0]:
shallowWindowedWells=sWellPPScenarios[sWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtSWellPPScenarios=sWellPPScenarios[sWellPPScenarios['ID'].isin(shallowWindowedWells)]
maxPressureList=[]
for ID in shallowWindowedWells:
  maxPressure=max(filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtSWellPPScenarios['MaxPressure']=maxPressureList
filtSWellPPScenarios=filtSWellPPScenarios.sort_values(by='MaxPressure',ascending=False)
filtSWellPPScenarios=filtSWellPPScenarios.merge(shallowPPWells,on='ID')

In [0]:
# Realization population results
# Total pressure change
# Number of wells
# 